## Toronto

In [1]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("Toronto")
print(location.address)
print((location.latitude, location.longitude))
print(location.raw)

Toronto, Golden Horseshoe, Ontario, M6K 1X9, Canada
(43.653963, -79.387207)
{'place_id': 111826, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 18063533, 'boundingbox': ['43.493963', '43.813963', '-79.547207', '-79.227207'], 'lat': '43.653963', 'lon': '-79.387207', 'display_name': 'Toronto, Golden Horseshoe, Ontario, M6K 1X9, Canada', 'class': 'place', 'type': 'city', 'importance': 0.8230149417022804, 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_place_city.p.20.png'}


C:\Users\croon\Anaconda3\envs\Work\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


In [2]:
from bs4 import BeautifulSoup
import requests 
import lxml.html as lh
import urllib3
import bs4 as bs
import urllib.request
import pandas as pd

In [3]:
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
import geopy
import geopandas

Read table with beautiful soup

In [102]:
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df=df[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Data Cleaning 

In [103]:
#drop all rows that has no attributes for Neighbourhood and Borough
df = df[~df['Borough'].isin(['Not assigned'])]
df = df[~df['Neighbourhood'].isin(['Not assigned'])]
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [104]:
df1=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()
df1.reset_index()
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Get coordinates

In [28]:
address = 'Toronto'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [29]:
CLIENT_ID = 'DV5DFTF4YGRFQD51YWF3OP10AEYMOJJX1MGWV0MLBEQUK0MC' # your Foursquare ID
CLIENT_SECRET = 'XJQ0Z2BDLT4Y2N1SRHYLFDKDRILBV3RRWMO5HWKTZG2RLXKF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DV5DFTF4YGRFQD51YWF3OP10AEYMOJJX1MGWV0MLBEQUK0MC
CLIENT_SECRET:XJQ0Z2BDLT4Y2N1SRHYLFDKDRILBV3RRWMO5HWKTZG2RLXKF


In [30]:
df1.loc[0,'Neighbourhood']

'Rouge,Malvern'

In [31]:
df1=df1.reset_index()

Get Latitude and longitude

In [41]:
from geopy.extra.rate_limiter import RateLimiter
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# 2- - create location column
df1['location'] = df1['Neighbourhood'].apply(geocode)

In [42]:
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df1['point'] = df1['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [43]:
# 4 - split point column into latitude, longitude and altitude columns
df1[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df1['point'].tolist(),index=df1.index)

In [44]:
df1=df1.drop(['location','point','altitude'],axis=1)
df1=df1.dropna()
df1.head()

,Unnamed: 0,index,Postcode,Borough,Neighbourhood,latitude,longitude
0,0,0,M1B,Scarborough,"Rouge,Malvern",43.809196,-79.221701
1,2,2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.753480,-79.179995
2,3,3,M1G,Scarborough,Woburn,42.479262,-71.152277
3,4,4,M1H,Scarborough,Cedarbrae,50.956318,-114.129323
4,5,5,M1J,Scarborough,Scarborough Village,43.743742,-79.211632


In [45]:
df1.to_csv('neigh_toronto')

In [46]:
df1=pd.read_csv('neigh_toronto')

In [47]:
neighbourhood_latitude = df1.loc[3, 'latitude'] # neighborhood latitude value
neighbourhood_longitude = df1.loc[3, 'longitude'] # neighborhood longitude value

neighbourhood_name = df1.loc[3, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Cedarbrae are 50.956318200000005, -114.12932350000001.


In [49]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL



'https://api.foursquare.com/v2/venues/explore?&client_id=DV5DFTF4YGRFQD51YWF3OP10AEYMOJJX1MGWV0MLBEQUK0MC&client_secret=XJQ0Z2BDLT4Y2N1SRHYLFDKDRILBV3RRWMO5HWKTZG2RLXKF&v=20180605&ll=43.653963,-79.387207&radius=500&limit=100'

In [50]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e4d771398205d001b03df4d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 74,
  'suggestedBounds': {'ne': {'lat': 43.6584630045, 'lng': -79.38099903084075},
   'sw': {'lat': 43.649462995499995, 'lng': -79.39341496915925}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          

In [51]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [52]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Japango,Sushi Restaurant,43.655268,-79.385165
2,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501
3,Rolltation,Japanese Restaurant,43.654918,-79.387424
4,Poke Guys,Poke Place,43.654895,-79.385052


In [53]:
nearby_venues.to_csv('nearby_venues')

In [54]:
df3=pd.read_csv('nearby_venues').drop('Unnamed: 0',axis=1)
df3.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Japango,Sushi Restaurant,43.655268,-79.385165
2,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501
3,Rolltation,Japanese Restaurant,43.654918,-79.387424
4,Poke Guys,Poke Place,43.654895,-79.385052


In [55]:
map1 = folium.Map(
    location=[43.653963,-79.387207],
    tiles='cartodbpositron',
    zoom_start=12,
)
df1.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

## Explore neighbourhoods

In [56]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [190]:
df4=pd.read_csv('neigh_toronto').drop(['Unnamed: 0','Unnamed: 0.1','index'],axis=1)
df4.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)
df4.head()

,Postcode,Borough,Neighborhood,latitude,longitude
0,M1B,Scarborough,"Rouge,Malvern",43.809196,-79.221701
1,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.753480,-79.179995
2,M1G,Scarborough,Woburn,42.479262,-71.152277
3,M1H,Scarborough,Cedarbrae,50.956318,-114.129324
4,M1J,Scarborough,Scarborough Village,43.743742,-79.211632


In [193]:
venues = getNearbyVenues(names=df4['Neighborhood'],
                                   latitudes=df4['latitude'],
                                   longitudes=df4['longitude']
                                  )



Rouge,Malvern
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
Agincourt
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Northwood Park,York University
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Rosedale
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Harbord,University of Toronto
Lawrence Heights,Lawrence Manor
Glencairn
Humewood-Cedarvale
Caledonia-Fairbanks
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
The Junction North,Runnymede
Hig

## Explore

In [192]:
print(venues.shape)
venues.head()

(1787, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.809196,-79.221701,Shoppers Drug Mart,43.809610,-79.222729,Pharmacy
1,"Rouge,Malvern",43.809196,-79.221701,Subway,43.806961,-79.221476,Sandwich Place
2,"Rouge,Malvern",43.809196,-79.221701,Malvern Arena,43.808594,-79.216634,Skating Rink
3,"Rouge,Malvern",43.809196,-79.221701,Pizza Hut,43.808326,-79.220616,Pizza Place
4,"Rouge,Malvern",43.809196,-79.221701,Pizza Pizza,43.806613,-79.221243,Pizza Place


In [194]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,1,1,1,1,1,1
"Alderwood,Long Branch",8,8,8,8,8,8
Bayview Village,11,11,11,11,11,11
Berczy Park,100,100,100,100,100,100
Caledonia-Fairbanks,11,11,11,11,11,11
...,...,...,...,...,...,...
Willowdale South,48,48,48,48,48,48
Willowdale West,48,48,48,48,48,48
Woburn,29,29,29,29,29,29


In [195]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 252 uniques categories.


In [196]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [197]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood,Long Branch",0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,Bayview Village,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,Berczy Park,0.000000,0.0,0.0,0.010000,0.0,0.02,0.0,0.0,0.0,...,0.0,0.010000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,Caledonia-Fairbanks,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Willowdale South,0.020833,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
57,Willowdale West,0.020833,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
58,Woburn,0.000000,0.0,0.0,0.034483,0.0,0.00,0.0,0.0,0.0,...,0.0,0.034483,0.0,0.0,0.034483,0.0,0.0,0.0,0.0,0.0
59,Woodbine Heights,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [198]:
grouped.shape

(61, 252)

print each neighbourhood with the most common venues

In [199]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                             venue  freq
0               Miscellaneous Shop   1.0
1                     Optical Shop   0.0
2       Modern European Restaurant   0.0
3  Molecular Gastronomy Restaurant   0.0
4              Monument / Landmark   0.0


----Alderwood,Long Branch----
            venue  freq
0     Pizza Place  0.25
1             Pub  0.12
2        Pharmacy  0.12
3    Skating Rink  0.12
4  Sandwich Place  0.12


----Bayview Village----
                 venue  freq
0       Breakfast Spot  0.09
1       Sandwich Place  0.09
2  Sporting Goods Shop  0.09
3          Gas Station  0.09
4          Fish Market  0.09


----Berczy Park----
                 venue  freq
0          Coffee Shop  0.09
1                 Café  0.06
2  Japanese Restaurant  0.04
3   Italian Restaurant  0.04
4               Bakery  0.04


----Caledonia-Fairbanks----
           venue  freq
0  Deli / Bodega  0.18
1           Bank  0.09
2     Donut Shop  0.09
3    Beer Garden  0.09
4   Gourmet Shop  0

## Put in a pandas dataframe

In [200]:
#function to sort the venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [201]:
#new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Miscellaneous Shop,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
1,"Alderwood,Long Branch",Pizza Place,Sandwich Place,Coffee Shop,Skating Rink,Gym,Pharmacy,Pub,Falafel Restaurant,Farmers Market,Dog Run
2,Bayview Village,Fish Market,Clothing Store,Fast Food Restaurant,Gas Station,Bank,Breakfast Spot,Sandwich Place,Outdoor Supply Store,Sporting Goods Shop,Pizza Place
3,Berczy Park,Coffee Shop,Café,Restaurant,Bakery,Japanese Restaurant,Hotel,Italian Restaurant,Seafood Restaurant,Beer Bar,Gym
4,Caledonia-Fairbanks,Deli / Bodega,Science Museum,Movie Theater,Bank,Donut Shop,Beer Garden,Gourmet Shop,Diner,Library,Pharmacy


In [202]:
# set number of clusters
kclusters = 5
grouped_clustering = grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 4, 4, 4, 4, 4, 4, 4, 4, 1])

In [203]:
# add clustering labels
neighborhoods_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)


In [204]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,Agincourt,Miscellaneous Shop,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
1,4,"Alderwood,Long Branch",Pizza Place,Sandwich Place,Coffee Shop,Skating Rink,Gym,Pharmacy,Pub,Falafel Restaurant,Farmers Market,Dog Run
2,4,Bayview Village,Fish Market,Clothing Store,Fast Food Restaurant,Gas Station,Bank,Breakfast Spot,Sandwich Place,Outdoor Supply Store,Sporting Goods Shop,Pizza Place
3,4,Berczy Park,Coffee Shop,Café,Restaurant,Bakery,Japanese Restaurant,Hotel,Italian Restaurant,Seafood Restaurant,Beer Bar,Gym
4,4,Caledonia-Fairbanks,Deli / Bodega,Science Museum,Movie Theater,Bank,Donut Shop,Beer Garden,Gourmet Shop,Diner,Library,Pharmacy


In [205]:
merged

,Postcode,Borough,Neighbourhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.809196,-79.221701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.753480,-79.179995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M1G,Scarborough,Woburn,42.479262,-71.152277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M1H,Scarborough,Cedarbrae,50.956318,-114.129324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M1J,Scarborough,Scarborough Village,43.743742,-79.211632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,M9L,North York,Humber Summit,43.760078,-79.571760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,M9M,North York,"Emery,Humberlea",43.719268,-79.537142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,M9N,York,Weston,51.347193,-2.977892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,M9P,Etobicoke,Westmount,45.485733,-73.596395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [208]:
merged = df4
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.809196,-79.221701,4.0,Fast Food Restaurant,Pizza Place,Pharmacy,Convenience Store,Grocery Store,Bubble Tea Shop,Skating Rink,Park,Sandwich Place,Gym / Fitness Center
1,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.753480,-79.179995,3.0,Park,Women's Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
2,M1G,Scarborough,Woburn,42.479262,-71.152277,4.0,Italian Restaurant,Pizza Place,Chinese Restaurant,South Indian Restaurant,Bank,Tapas Restaurant,Beer Store,Bowling Alley,Brazilian Restaurant,Liquor Store
3,M1H,Scarborough,Cedarbrae,50.956318,-114.129324,4.0,Circus,Food Truck,Bar,Vietnamese Restaurant,Women's Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Fast Food Restaurant
4,M1J,Scarborough,Scarborough Village,43.743742,-79.211632,4.0,Coffee Shop,Gym,Pub,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Exhibit,Empanada Restaurant,English Restaurant,Ethiopian Restaurant


In [212]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['latitude'], merged['longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Exploring Clusters

In [213]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,North York,0.0,Pool,Women's Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [ ]:
#cluster 2

In [214]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Downtown Toronto,1.0,Athletics & Sports,Home Service,Café,Fast Food Restaurant,Women's Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Farmers Market
56,West Toronto,1.0,Fast Food Restaurant,Women's Store,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Farmers Market,Falafel Restaurant


In [215]:
#cluster 3

In [216]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,2.0,Miscellaneous Shop,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [218]:
#cluster 4 

In [217]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3.0,Park,Women's Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [219]:
#cluster 5

In [220]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4.0,Fast Food Restaurant,Pizza Place,Pharmacy,Convenience Store,Grocery Store,Bubble Tea Shop,Skating Rink,Park,Sandwich Place,Gym / Fitness Center
2,Scarborough,4.0,Italian Restaurant,Pizza Place,Chinese Restaurant,South Indian Restaurant,Bank,Tapas Restaurant,Beer Store,Bowling Alley,Brazilian Restaurant,Liquor Store
3,Scarborough,4.0,Circus,Food Truck,Bar,Vietnamese Restaurant,Women's Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Fast Food Restaurant
4,Scarborough,4.0,Coffee Shop,Gym,Pub,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Exhibit,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
6,Scarborough,4.0,Caribbean Restaurant,Athletics & Sports,Chinese Restaurant,Coffee Shop,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Women's Store
7,Scarborough,4.0,French Restaurant,Ice Cream Shop,Bar,Café,Plaza,Coffee Shop,Hotel,Park,Tea Room,Wine Bar
8,North York,4.0,Pool,Coffee Shop,Shopping Mall,Korean Restaurant,Sandwich Place,Fast Food Restaurant,Restaurant,Recreation Center,Park,Bank
9,North York,4.0,Coffee Shop,Furniture / Home Store,Scandinavian Restaurant,Train Station,Metro Station,Food & Drink Shop,Ice Cream Shop,Park,English Restaurant,Ethiopian Restaurant
10,North York,4.0,Fish Market,Clothing Store,Fast Food Restaurant,Gas Station,Bank,Breakfast Spot,Sandwich Place,Outdoor Supply Store,Sporting Goods Shop,Pizza Place
12,North York,4.0,Coffee Shop,Korean Restaurant,Hardware Store,Discount Store,Middle Eastern Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Sushi Restaurant,Restaurant,Sandwich Place


One can see that the choice of 5 clusters is maybe not the best, it could probabily be improved